# Disagreement in Crowdsourced Policymaking

## Regression analysis

This notebook contains the regression analysis conducted for the article *The Value of Disagreement in Crowdsourced Policymaking:Idea Generation Through Elaborated Perspectives*. The analysis is splitted in two parts. The first part aims at studying the factors that influence disagreement while the second part presents the analysis of the factors that influence the generation of news ideas.

After loading the data and importing the required technical libraries and tools, the analyses is presented step by step.

### Load libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

from collections import defaultdict
from predictor import predict_disagreement, get_dataset
from statsmodels.formula.api import glm, ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

plt.style.use('seaborn')

ModuleNotFoundError: No module named 'statsmodels'